In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests   
import shutil      
import datetime
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import os
import winsound
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')
import warnings
warnings.filterwarnings('ignore')

from selenium.common.exceptions import WebDriverException

In [20]:
# Log in to Rotowire
driver = webdriver.Chrome()
login_url = 'https://www.rotowire.com/users/login.php'
username = os.environ.get('rot_login')
password = os.environ.get('COMMON_PWORD')

# go to login page
driver.get(login_url)
# enter username
username_field = driver.find_element_by_name('username')
username_field.send_keys(username)
# enter password
password_field = driver.find_element_by_name('password')
password_field.send_keys(password)
# click login button
login_xpath = '/html/body/div[1]/div/main/div/div[1]/form/button'

# click login button
login_button = driver.find_element_by_xpath(login_xpath)
login_button.click()

# wait 3 seconds for page to load
time.sleep(3)

pg_url = 'https://www.rotowire.com/daily/nba/defense-vspos.php'
driver.get(pg_url)

# download the csv
soup = BeautifulSoup(driver.page_source, 'html.parser')
# wait 2 seconds
time.sleep(3)

# csv_link = soup.find('button', class_ = 'export-button is-csv hovering')
# csv_link.click()

csv_xpath = '//*[@id="NBAPlayers"]/div[3]/div[2]/button[2]'
csv_link = driver.find_element_by_xpath(csv_xpath)
csv_link.click()
# close browser
driver.close()

In [21]:
# now, 'rotowire-NBA-defense-vs-pos' is in the downloads folder

d_vs_position_files = pd.DataFrame()

# check files in folder that match
folder = os.listdir('C:\\Users\\Travis\\Downloads')
for file in folder:
    if file.startswith('rotowire-NBA-defense-vs-pos'):
        # add to dataframe
        d_vs_position_files = d_vs_position_files.append({'file_name': file}, ignore_index = True)

d_vs_position_files

,file_name
0,rotowire-NBA-defense-vs-pos (1).csv
1,rotowire-NBA-defense-vs-pos (2).csv
2,rotowire-NBA-defense-vs-pos (3).csv
3,rotowire-NBA-defense-vs-pos (4).csv
4,rotowire-NBA-defense-vs-pos (5).csv
5,rotowire-NBA-defense-vs-pos (5).csv.crdownload
6,rotowire-NBA-defense-vs-pos (6).csv
7,rotowire-NBA-defense-vs-pos (6).csv (1).crdown...
8,rotowire-NBA-defense-vs-pos (6).csv.crdownload
9,rotowire-NBA-defense-vs-pos (7).csv


In [22]:
# identify file with highest number
d_vs_position_files['file_number'] = d_vs_position_files['file_name'].str[-6:-5]
# if file_number is not a number, make it 0
d_vs_position_files['file_number'] = d_vs_position_files['file_number'].apply(lambda x: 0 if x.isnumeric() == False else x)
d_vs_position_files['file_number'] = d_vs_position_files['file_number'].astype(int)
# select file with highest number
newest_file = d_vs_position_files[d_vs_position_files['file_number'] == d_vs_position_files['file_number'].max()]
newest_file = newest_file['file_name'].values
# read file
d_vs_position = pd.read_csv('C:\\Users\\Travis\\Downloads\\' + newest_file[0])
d_vs_position

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%
0,SAS,PG,27.2,14.3,3.1,4.1,1.0,0.3,1.5,1.9,49.6,81.3
1,GSW,PG,26.8,13.2,3.5,4.4,1.0,0.3,1.6,1.6,44.9,81.1
2,SAC,PG,26.5,13.9,3.2,4.1,1.0,0.2,1.8,1.6,46.7,85.2
3,MIN,PG,26.4,13.5,3.5,4.2,0.9,0.2,1.9,1.7,44.4,80.7
4,ATL,C,26.4,11.7,7.2,1.8,0.7,0.9,1.3,0.6,59.3,77.6
...,...,...,...,...,...,...,...,...,...,...,...,...
145,SAS,SF,19.6,10.3,3.1,2.0,0.8,0.3,1.1,1.3,49.7,77.7
146,BOS,C,19.5,8.6,5.9,1.5,0.3,0.5,1.2,0.5,55.4,69.8
147,PHX,SF,19.0,9.7,3.4,1.8,0.8,0.4,1.2,1.1,45.6,78.4
148,NYK,C,18.9,8.3,5.3,1.4,0.4,0.6,1.0,0.5,53.3,68.5


In [23]:
today = datetime.date.today()
today = today.strftime('%Y-%m-%d')
today

'2023-01-17'

In [24]:
d_vs_position.to_csv('data/team/aggregates/daily_updates/defense_vs_position' + str(today) + '.csv', index = False)

In [25]:
# load big odds df for today

big_odds_files = os.listdir('data/team/aggregates/daily_updates')
big_odds_files = [file for file in big_odds_files if today in file]
big_odds_files = [file for file in big_odds_files if 'games_and_odds' in file]
big_odds_file = big_odds_files[0]
big_odds_file

'games_and_odds_2023-01-17_07-59.csv'

In [26]:
team_name_dict= [{'team': 'Atlanta', 'abbreviation': 'ATL'},
                 {'team': 'Boston', 'abbreviation': 'BOS'},
                 {'team': 'Brooklyn', 'abbreviation': 'BKN'},
                 {'team': 'Charlotte', 'abbreviation': 'CHA'},
                 {'team': 'Chicago', 'abbreviation': 'CHI'},
                 {'team': 'Cleveland', 'abbreviation': 'CLE'},
                 {'team': 'Dallas', 'abbreviation': 'DAL'},
                 {'team': 'Denver', 'abbreviation': 'DEN'},
                 {'team': 'Detroit', 'abbreviation': 'DET'},
                 {'team': 'Golden State', 'abbreviation': 'GSW'},
                 {'team': 'Houston', 'abbreviation': 'HOU'},
                 {'team': 'Indiana', 'abbreviation': 'IND'},
                 {'team': 'LA Clippers', 'abbreviation': 'LAC'},
                 {'team': 'LA Lakers', 'abbreviation': 'LAL'},
                 {'team': 'Memphis', 'abbreviation': 'MEM'},
                 {'team': 'Miami', 'abbreviation': 'MIA'},
                 {'team': 'Milwaukee', 'abbreviation': 'MIL'},
                 {'team': 'Minnesota', 'abbreviation': 'MIN'},
                 {'team': 'New Orleans', 'abbreviation': 'NOP'},
                 {'team': 'New York', 'abbreviation': 'NYK'},
                 {'team': 'Oklahoma City', 'abbreviation': 'OKC'},
                 {'team': 'Orlando', 'abbreviation': 'ORL'},
                 {'team': 'Philadelphia', 'abbreviation': 'PHI'},
                 {'team': 'Phoenix', 'abbreviation': 'PHX'},
                 {'team': 'Portland', 'abbreviation': 'POR'},
                 {'team': 'Sacramento', 'abbreviation': 'SAC'},
                 {'team': 'San Antonio', 'abbreviation': 'SAS'},
                 {'team': 'Toronto', 'abbreviation': 'TOR'},
                 {'team': 'Utah', 'abbreviation': 'UTA'},
                 {'team': 'Washington', 'abbreviation': 'WAS'}]

In [27]:
# get average for all teams by position
d_vs_position_avg = d_vs_position.groupby('POS').mean().reset_index()
d_vs_position_avg

,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%
0,C,22.246667,9.663333,6.016667,1.710000,0.530000,0.766667,1.250000,0.506667,55.376667,72.216667
1,PF,22.273333,10.816667,4.910000,1.843333,0.650000,0.586667,1.216667,1.083333,48.840000,78.360000
2,PG,24.610000,12.413333,3.073333,4.003333,0.856667,0.270000,1.633333,1.523333,44.150000,82.443333
3,SF,20.876667,10.926667,3.683333,2.003333,0.736667,0.380000,1.230000,1.340000,46.140000,79.626667
4,SG,22.263333,11.993333,3.150000,2.676667,0.780000,0.283333,1.390000,1.716667,44.156667,81.176667


In [28]:
# Rank each team by position for PTS
d_vs_position['PTS_rank'] = d_vs_position.groupby('POS')['PTS'].rank(ascending = False)
d_vs_position['PTS_rank'] = d_vs_position['PTS_rank'].astype(int)
d_vs_position

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
0,SAS,PG,27.2,14.3,3.1,4.1,1.0,0.3,1.5,1.9,49.6,81.3,1
1,GSW,PG,26.8,13.2,3.5,4.4,1.0,0.3,1.6,1.6,44.9,81.1,4
2,SAC,PG,26.5,13.9,3.2,4.1,1.0,0.2,1.8,1.6,46.7,85.2,2
3,MIN,PG,26.4,13.5,3.5,4.2,0.9,0.2,1.9,1.7,44.4,80.7,3
4,ATL,C,26.4,11.7,7.2,1.8,0.7,0.9,1.3,0.6,59.3,77.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,SAS,SF,19.6,10.3,3.1,2.0,0.8,0.3,1.1,1.3,49.7,77.7,26
146,BOS,C,19.5,8.6,5.9,1.5,0.3,0.5,1.2,0.5,55.4,69.8,27
147,PHX,SF,19.0,9.7,3.4,1.8,0.8,0.4,1.2,1.1,45.6,78.4,30
148,NYK,C,18.9,8.3,5.3,1.4,0.4,0.6,1.0,0.5,53.3,68.5,29


In [29]:
big_odds_df = pd.read_csv('data/team/aggregates/daily_updates/'+big_odds_file)
big_odds_df['team1_abv'] = big_odds_df['team1'].map(lambda x: [d['abbreviation'] for d in team_name_dict if d['team'] == x][0])
big_odds_df['team2_abv'] = big_odds_df['team2'].map(lambda x: [d['abbreviation'] for d in team_name_dict if d['team'] == x][0])
big_odds_df

,Unnamed: 0,team1,team2,time,opening_spread_t1,opening_spread_t2,current_spread_t1,current_spread_t2,opening_moneyline_t1,opening_moneyline_t2,current_moneyline_t1,current_moneyline_t2,team1_abv,team2_abv
0,0,Toronto,Milwaukee,16:30:00,6.0,-6.0,4.5,-4.5,205,-250,160,-180,TOR,MIL
1,1,Brooklyn,San Antonio,17:00:00,-5.0,5.0,-5.5,5.5,-227,160,-215,185,BKN,SAS
2,2,Portland,Denver,18:00:00,8.5,-8.5,6.5,-6.5,285,-455,217,-257,POR,DEN


#### Game 1

In [30]:
away_team_1 = big_odds_df['team1_abv'].values[0]
at1 = d_vs_position[d_vs_position['TEAM'] == away_team_1]
at1

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
5,TOR,PG,25.7,12.9,3.3,4.4,0.9,0.3,2.2,1.5,46.0,82.5,9
47,TOR,C,23.4,10.5,6.6,1.5,0.3,0.8,1.5,0.6,57.0,73.4,7
77,TOR,PF,22.1,10.7,5.2,1.8,0.6,0.6,1.6,1.1,49.7,77.8,18
101,TOR,SG,21.4,11.2,3.2,3.0,0.6,0.2,1.5,1.6,45.5,82.8,24
138,TOR,SF,20.0,11.0,3.3,2.0,0.5,0.4,1.3,1.4,47.9,83.1,12


In [31]:
home_team_1 = big_odds_df['team2_abv'].values[0]
ht1 = d_vs_position[d_vs_position['TEAM'] == home_team_1]
ht1

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
34,MIL,PG,24.0,12.7,3.1,3.4,0.8,0.3,1.3,1.5,44.6,81.4,12
90,MIL,C,21.5,8.9,5.8,1.7,0.6,0.7,1.1,0.6,49.4,72.1,22
121,MIL,SG,20.8,11.2,3.3,2.4,0.7,0.1,1.2,1.5,42.7,82.1,24
122,MIL,PF,20.8,10.1,4.6,1.7,0.7,0.5,1.2,1.1,46.8,83.6,26
137,MIL,SF,20.0,10.8,3.6,1.8,0.6,0.4,1.2,1.1,45.1,78.1,16


#### Game 2

In [32]:
away_team_2 = big_odds_df['team1_abv'].values[1]
at2 = d_vs_position[d_vs_position['TEAM'] == away_team_2]
at2

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
9,BKN,PG,25.4,12.5,3.3,4.0,1.1,0.2,1.6,1.7,42.5,83.6,14
20,BKN,SG,24.8,13.7,3.1,2.9,0.8,0.3,1.4,2.3,45.0,80.4,2
102,BKN,PF,21.3,10.4,5.1,1.5,0.7,0.5,1.3,0.8,47.6,76.2,22
112,BKN,SF,21.0,11.2,3.7,1.9,0.7,0.4,1.2,1.4,44.0,78.0,10
139,BKN,C,20.0,9.0,5.7,1.3,0.5,0.6,1.1,0.5,50.6,65.6,19


In [33]:
home_team_2 = big_odds_df['team2_abv'].values[1]
ht2 = d_vs_position[d_vs_position['TEAM'] == home_team_2]
ht2

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
0,SAS,PG,27.2,14.3,3.1,4.1,1.0,0.3,1.5,1.9,49.6,81.3,1
18,SAS,C,24.9,11.2,6.5,2.0,0.5,0.8,1.4,0.5,57.9,70.9,3
91,SAS,PF,21.5,10.6,4.6,1.8,0.7,0.5,1.1,1.0,52.9,79.9,20
107,SAS,SG,21.2,11.2,2.9,2.6,0.9,0.3,1.2,1.5,44.6,79.5,24
145,SAS,SF,19.6,10.3,3.1,2.0,0.8,0.3,1.1,1.3,49.7,77.7,26


#### Game 3

In [34]:
away_team_3 = big_odds_df['team1_abv'].values[2]
at3 = d_vs_position[d_vs_position['TEAM'] == away_team_3]
at3

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
39,POR,PG,23.8,11.8,3.1,4.2,0.8,0.2,1.7,1.4,43.7,83.1,25
93,POR,SG,21.5,11.6,3.1,2.4,0.9,0.2,1.4,1.7,43.7,78.9,20
109,POR,C,21.1,9.7,5.3,1.7,0.5,0.7,1.1,0.5,55.3,68.6,15
120,POR,SF,20.8,10.4,3.7,2.2,0.9,0.3,1.1,1.2,45.6,78.5,24
140,POR,PF,19.9,10.1,4.1,1.6,0.6,0.4,1.1,0.9,48.5,75.8,26


In [35]:
home_team_3 = big_odds_df['team2_abv'].values[2]
ht3 = d_vs_position[d_vs_position['TEAM'] == home_team_3]
ht3

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
11,DEN,PG,25.4,12.9,2.9,4.7,0.7,0.2,1.6,1.5,44.8,82.9,9
110,DEN,PF,21.0,10.0,4.3,1.8,0.8,0.6,1.0,1.0,51.8,72.5,28
111,DEN,SG,21.0,11.2,2.8,2.7,0.8,0.3,1.4,1.6,44.1,78.6,24
143,DEN,SF,19.7,10.0,3.2,2.1,0.8,0.4,1.0,1.3,46.0,73.6,29
144,DEN,C,19.7,8.6,5.0,1.5,0.6,0.7,1.0,0.4,52.9,69.3,27


#### Game 4

In [36]:
away_team_4 = big_odds_df['team1_abv'].values[3]
at4 = d_vs_position[d_vs_position['TEAM'] == away_team_4]
at4

IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
home_team_4 = big_odds_df['team2_abv'].values[3]
ht4 = d_vs_position[d_vs_position['TEAM'] == home_team_4]
ht4

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
22,NYK,PG,24.7,12.1,3.3,4.0,0.7,0.4,1.5,1.5,42.2,80.1,19
98,NYK,SG,21.4,11.7,3.1,2.6,0.5,0.3,1.3,1.7,44.4,80.7,18
132,NYK,SF,20.2,10.1,3.9,2.0,0.7,0.3,1.1,1.2,43.9,78.3,28
141,NYK,PF,19.7,9.7,4.2,1.7,0.5,0.6,1.1,1.1,47.7,78.0,30
148,NYK,C,18.9,8.3,5.3,1.4,0.4,0.6,1.0,0.5,53.7,67.8,29


#### Game 5

In [ ]:
away_team_5 = big_odds_df['team1_abv'].values[4]
at5 = d_vs_position[d_vs_position['TEAM'] == away_team_5]
at5

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
1,GSW,PG,26.8,13.3,3.5,4.4,1.0,0.3,1.7,1.6,44.8,80.8,4
15,GSW,C,25.1,11.3,6.7,1.9,0.7,0.7,1.7,0.6,53.9,72.2,2
30,GSW,SG,24.1,12.8,3.3,3.0,0.9,0.3,1.4,1.8,43.8,81.2,7
81,GSW,PF,22.0,11.1,5.0,1.6,0.6,0.4,1.3,1.2,47.8,74.3,11
105,GSW,SF,21.2,11.3,3.5,2.1,0.8,0.3,1.0,1.4,46.2,77.9,9


In [ ]:
home_team_5 = big_odds_df['team2_abv'].values[4]
ht5 = d_vs_position[d_vs_position['TEAM'] == home_team_5]
ht5

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
45,WAS,PG,23.5,11.6,3.0,3.9,0.8,0.3,1.5,1.3,43.1,80.7,25
51,WAS,SG,23.2,12.7,3.5,2.6,0.8,0.3,1.3,1.7,45.0,82.6,8
82,WAS,SF,21.8,12.0,3.7,2.0,0.6,0.4,1.0,1.6,46.2,86.1,2
93,WAS,PF,21.5,10.7,4.8,1.7,0.5,0.6,1.0,1.2,46.4,80.9,18
112,WAS,C,21.0,9.8,5.5,1.3,0.4,0.7,1.1,0.5,52.0,78.3,13


#### Game 6

In [ ]:
away_team_6 = big_odds_df['team1_abv'].values[5]
at6 = d_vs_position[d_vs_position['TEAM'] == away_team_6]
at6

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
21,MIA,SG,24.8,13.1,3.6,3.2,0.8,0.3,1.8,2.1,45.3,78.3,3
35,MIA,SF,23.9,12.4,4.4,2.5,0.8,0.4,1.7,1.6,49.6,77.7,1
47,MIA,PG,23.4,11.5,3.1,4.0,0.8,0.2,1.8,1.7,42.5,82.8,26
90,MIA,PF,21.5,10.1,5.1,1.9,0.6,0.5,1.3,1.0,48.4,74.0,26
143,MIA,C,19.7,8.2,5.8,1.5,0.5,0.6,1.3,0.4,56.9,72.2,30


In [ ]:
home_team_6 = big_odds_df['team2_abv'].values[5]
ht6 = d_vs_position[d_vs_position['TEAM'] == home_team_6]
ht6

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
4,ATL,C,26.0,11.6,7.1,1.8,0.6,0.9,1.3,0.6,59.6,77.4,1
27,ATL,PF,24.3,11.6,5.4,2.0,0.6,0.7,1.2,1.1,46.6,85.4,2
43,ATL,PG,23.6,11.2,3.2,4.1,0.9,0.3,1.6,1.1,42.7,82.9,27
66,ATL,SF,22.5,11.8,4.0,2.0,0.8,0.4,1.3,1.3,45.1,82.0,4
115,ATL,SG,20.9,11.0,3.3,2.6,0.7,0.2,1.5,1.5,42.9,81.9,28


#### Game 7

In [ ]:
away_team_7 = big_odds_df['team1_abv'].values[6]
at7 = d_vs_position[d_vs_position['TEAM'] == away_team_7]
at7

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
33,UTA,PG,24.0,12.6,3.0,3.5,0.8,0.3,1.6,1.5,45.5,80.2,12
34,UTA,SG,24.0,13.7,3.3,2.5,0.8,0.3,1.3,1.7,45.0,83.9,1
39,UTA,C,23.7,10.7,6.2,1.8,0.5,0.9,1.4,0.4,56.5,74.1,5
123,UTA,PF,20.6,9.9,4.5,1.7,0.7,0.6,1.1,0.8,48.9,77.9,29
139,UTA,SF,19.8,10.4,3.4,1.7,0.9,0.3,1.0,1.2,45.9,77.1,23


In [ ]:
home_team_7 = big_odds_df['team2_abv'].values[6]
ht7 = d_vs_position[d_vs_position['TEAM'] == home_team_7]
ht7

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
2,MIN,PG,26.5,13.5,3.5,4.3,0.9,0.2,1.9,1.7,44.3,80.5,3
40,MIN,SG,23.7,12.6,3.4,2.9,0.9,0.2,1.6,1.8,45.9,77.5,9
55,MIN,PF,23.0,11.5,4.7,1.8,0.9,0.5,1.2,1.4,46.7,77.9,4
88,MIN,SF,21.7,11.4,3.8,2.2,0.7,0.3,1.2,1.6,46.6,81.6,7
100,MIN,C,21.3,8.7,5.6,1.9,0.7,0.8,1.4,0.6,50.6,66.1,24


#### Game 8

In [ ]:
away_team_8 = big_odds_df['team1_abv'].values[7]
at8 = d_vs_position[d_vs_position['TEAM'] == away_team_8]
at8

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
28,PHX,C,24.3,10.4,6.7,2.0,0.7,0.7,1.5,0.5,56.4,78.9,8
69,PHX,PF,22.4,10.9,4.9,1.9,0.8,0.5,1.3,1.0,51.7,77.6,15
74,PHX,SG,22.1,12.1,3.1,2.6,0.8,0.3,1.6,1.6,42.7,82.8,11
75,PHX,PG,22.1,11.9,2.7,3.3,0.7,0.2,1.5,1.4,43.5,83.0,21
146,PHX,SF,19.2,9.7,3.4,1.8,0.8,0.4,1.2,1.1,45.6,78.6,30


In [ ]:
home_team_8 = big_odds_df['team2_abv'].values[7]
ht8 = d_vs_position[d_vs_position['TEAM'] == home_team_8]
ht8

,TEAM,POS,FPTS ALLOWED,PTS,REB,AST,STL,BLK,TO,3PM,FG%,FT%,PTS_rank
44,MEM,PF,23.5,11.4,4.8,2.1,0.6,0.8,1.3,1.2,46.2,81.8,5
48,MEM,PG,23.3,11.2,3.4,3.9,0.7,0.2,1.5,1.5,41.0,82.1,27
77,MEM,SG,22.0,11.8,3.3,2.6,0.8,0.3,1.3,1.6,44.0,80.2,17
97,MEM,C,21.4,8.9,5.5,1.9,0.6,0.9,1.3,0.5,52.1,69.4,21
118,MEM,SF,20.9,10.7,3.7,2.3,0.7,0.4,1.4,1.3,44.0,79.6,19


#### Game 9

In [ ]:
away_team_9 = big_odds_df['team1_abv'].values[8]
at9 = d_vs_position[d_vs_position['TEAM'] == away_team_9]
at9

IndexError: index 8 is out of bounds for axis 0 with size 8

In [ ]:
home_team_9 = big_odds_df['team2_abv'].values[8]
ht9 = d_vs_position[d_vs_position['TEAM'] == home_team_9]
ht9

IndexError: index 8 is out of bounds for axis 0 with size 8

#### Game 10

In [ ]:
# TODO: Scrape Draftkings PLayer Props

driver = None
if driver== None:
    driver = webdriver.Chrome()

url = 'https://sportsbook.draftkings.com/leagues/basketball/nba'
driver.get(url)

# wait 3 seconds for page to load
time.sleep(3)
body = driver.find_elements_by_class_name('sportsbook-league-page__body')
# find sgp buttons
sgp_buttons = driver.find_elements_by_class_name('toggle-sgp-badge__nav-link')
# get links from sgps
sgp_links = []
for button in sgp_buttons:
    sgp_links.append(button.get_attribute('href'))

sgp_links

sgp_links = pd.DataFrame(sgp_links, columns = ['link'])
# drop duplicates
sgp_links = sgp_links.drop_duplicates()
sgp_links


,link
0,https://sportsbook.draftkings.com/event/bos-ce...
2,https://sportsbook.draftkings.com/event/ind-pa...
4,https://sportsbook.draftkings.com/event/no-pel...
6,https://sportsbook.draftkings.com/event/tor-ra...
8,https://sportsbook.draftkings.com/event/gs-war...
10,https://sportsbook.draftkings.com/event/mia-he...
12,https://sportsbook.draftkings.com/event/uta-ja...
14,https://sportsbook.draftkings.com/event/pho-su...
16,https://sportsbook.draftkings.com/event/hou-ro...


In [ ]:
# get to fighter points page
driver.get(sgp_links['link'].values[0])
time.sleep(3)
points_page_xpath = '//*[@id="dom_SameGameParlayWeb"]/sb-comp/div/div/div[1]/div/div/button[4]'
driver.find_element_by_xpath(points_page_xpath).click()
time.sleep(3)
